In [10]:
import numpy as np
import tables

In [11]:
class Photometry(tables.IsDescription):
    JD         = tables.Float64Col()
    mag        = tables.Float64Col()
    magErr     = tables.Float64Col()
    magLimit   = tables.Float64Col()
    filterName = tables.StringCol(16)
    source     = tables.StringCol(32)

In [17]:
h5file = tables.open_file("iPTF16abcLightCurve.h5", mode="w")

In [18]:
tbl = h5file.create_table(h5file.root, "LightCurve", Photometry, "iPTF16abc Light Curve Table")
phot = tbl.row

In [19]:
with open("rawData.txt", "r") as fp:
    for row in fp:
        if len(row) < 5:
            continue
        if row.startswith("date"): 
            continue
        tmp = row[:-1].split(",")
        JD = float(tmp[1])
        filterName = tmp[2][1:-1]
        mag = float(tmp[4])
        magErr = float(tmp[5])
        magLimit = float(tmp[6])
        instrument = tmp[7][1:-1]
        reducedby = tmp[8][1:-1]
        if ((instrument.find("P48") > -1) and (reducedby.find("PTFIDE") > -1)) or\
           ((instrument.find("P60") > -1) and (reducedby.find("Fremling") > -1)) or\
           ((instrument.find("RATIR") > -1) and (reducedby.find("Fox") > -1)):
            phot["JD"] = JD
            phot["mag"] = mag if mag < 30 else None
            phot["magErr"] = magErr if magErr < 30 else None
            phot["magLimit"] = magLimit if magLimit < 30 else None
            phot["filterName"] = filterName
            phot["source"] = instrument
            phot.append()

In [20]:
import datetime 

def date2JD(date):
    a = (14 - date.month) / 12
    y = date.year + 4800 - a
    m = date.month + 12*a - 3
    jdn = date.day + (153*m + 2) / 5 + 365 * y + y/4 - y/100 + y/400 - 32045
    jd = jdn + (date.hour - 12) / 24. + date.minute / 1440. + date.second / 86400.
    return jd

with open("swiftRawData.txt", "r") as fp:
    for row in fp:
        if row.startswith("#"):
            continue
        if len(row) < 5:
            continue
        tmp = row[:-1].split("\t")
        date = datetime.datetime.strptime(tmp[1].split(".")[0], "%Y-%m-%d %H:%M:%S")
        JD = date2JD(date)
        mag = float(tmp[10])
        magErr = float(tmp[0])
        magLimit = float(tmp[7])
        filterName = tmp[8].strip()
        instrument = "Swift+UVOT"
        phot["JD"] = JD
        phot["mag"] = mag
        phot["magErr"] = magErr
        phot["magLimit"] = magLimit
        phot["filterName"] = filterName
        phot["source"] = instrument
        phot.append()

In [21]:
h5file.close()